In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-dpofwc01
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-dpofwc01
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=73bf5d828e3c4ba4e5c4b3c85bebc1323c332ce12270b99f84f2444ecb5a9cbf
  Stored in directory: /tmp/pip-ephem-wheel-cache-m3u4f8rn/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [23]:
import os
import torch
import clip
import numpy as np
import pandas as pd
import pyarrow.feather as feather
from PIL import Image
from urllib import request
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
device

'cuda'

In [25]:
import json

with open('/content/skin_cancer_dataset.json') as user_file:
    ds = user_file.read()
    
parsed_json = json.loads(ds)
print(parsed_json)

[{'id': 0, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png', 'cancer_img_caption': 'An open sore that does not heal', 'caption_speech': 'id_0.mp3'}, {'id': 1, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-1.jpg', 'cancer_img_caption': 'An open sore that bleeds, oozes, or crusts and remains open for a few weeks, only to heal up and then bleed again. ', 'caption_speech': 'id_1.mp3'}, {'id': 2, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-2.jpg', 'cancer_img_caption': 'A reddish patch or irritated area', 'caption_speech': 'id_2.mp3'}, {'id': 3, 'cancer_type': 'Basal Cell Carcinoma (BCC)', 'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-1-shiny-bump-340x240.png', 'cancer_img_caption': 'A shiny bump or nodule', 'cap

In [26]:
parsed_json

[{'id': 0,
  'cancer_type': 'Basal Cell Carcinoma (BCC)',
  'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png',
  'cancer_img_caption': 'An open sore that does not heal',
  'caption_speech': 'id_0.mp3'},
 {'id': 1,
  'cancer_type': 'Basal Cell Carcinoma (BCC)',
  'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-1.jpg',
  'cancer_img_caption': 'An open sore that bleeds, oozes, or crusts and remains open for a few weeks, only to heal up and then bleed again. ',
  'caption_speech': 'id_1.mp3'},
 {'id': 2,
  'cancer_type': 'Basal Cell Carcinoma (BCC)',
  'cancer_img': 'https://www.skincancer.org/wp-content/uploads/BCC-slider-2.jpg',
  'cancer_img_caption': 'A reddish patch or irritated area',
  'caption_speech': 'id_2.mp3'},
 {'id': 3,
  'cancer_type': 'Basal Cell Carcinoma (BCC)',
  'cancer_img': 'https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-1-shiny-bump-340x240.png',
  'cancer_img_caption

In [27]:
tkns,final_labels = [],[]
for i,ele in enumerate(parsed_json):
    print(f"{i}  -> {ele['cancer_img_caption']}")
    tkns.append(ele['cancer_img_caption'])
    final_labels.append(ele['cancer_type'])

0  -> An open sore that does not heal
1  -> An open sore that bleeds, oozes, or crusts and remains open for a few weeks, only to heal up and then bleed again. 
2  -> A reddish patch or irritated area
3  -> A shiny bump or nodule
4  -> White, yellow or waxy scar
5  -> A basal cell carcinoma may be pigmented on skin of color. Photo: Andrew Alexis, MD, MPH
6  -> BCC on the nose.
7  -> A small pink growth with a slightly raised, rolled edge and a crusted indentation in the center
8  -> A shiny bump or nodule 
9  -> Pigmented BCC behind the ear. Photo: Hugh Gloster, MD
10  -> A scar-like area that is white, yellow or waxy, and often has poorly defined borders
11  -> Pigmented BCC. Photo credit: Hugh Gloster, MD
12  -> BCC on the nose. Photo: Hugh Gloster, MD
13  -> A pink growth
14  -> A subtle pearly lesion in the middle of the lower eyelid’s lash line is a BCC.
15  -> A persistent, scaly red patch with irregular borders that sometimes crusts or bleeds.
16  -> An elevated growth with a cen

In [28]:
final_labels

['Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Basal Cell Carcinoma (BCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Squamous Cell Carcinoma (SCC)',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',
 'Melanoma',


In [29]:
len(set(final_labels))

4

In [30]:
import requests
img_ids = []
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.9"
           }

for i,ele in enumerate(parsed_json):
    print(f"{i}  -> {ele['cancer_img']}")
    img_ids.append(i)
    image_url = ele['cancer_img']
    img_data = requests.get(url=image_url, headers=headers).content
    with open(f'{i}.jpg', 'wb') as handler:
        handler.write(img_data)


0  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-2-open-sore-340x240.png
1  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-1.jpg
2  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-2.jpg
3  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-1-shiny-bump-340x240.png
4  -> https://www.skincancer.org/wp-content/uploads/basal-cell-carcinoma-4-scar-340x240.png
5  -> https://www.skincancer.org/wp-content/uploads/BCC_pigmented_Fitz_type_4_jpg_Bwr4Nu1C-340x240.jpg
6  -> https://www.skincancer.org/wp-content/uploads/bcclatinoman_sm.jpg
7  -> https://www.skincancer.org/wp-content/uploads/BCC_PinkGrowth-1-340x240.jpg
8  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-4.jpg
9  -> https://www.skincancer.org/wp-content/uploads/Pigmented-BCC-behind-the-ear-340x240.jpeg
10  -> https://www.skincancer.org/wp-content/uploads/BCC-slider-5.jpg
11  -> https://www.skincancer.org/wp-content/uploads/Pigmented-BCC-340x240.jpeg
12  ->

In [31]:
labels = ["Basal Cell Carcinoma (BCC)","Squamous Cell Carcinoma (SCC)","Melanoma", "Merkel Cell Carcinoma (MCC)",
              "Actinic Keratosis"]
text = clip.tokenize(tkns).to(device)

In [32]:
text.shape

torch.Size([48, 77])

In [33]:
# image embeddings
images = [
        preprocess(
            Image.open(f"/content/{img_id}.jpg")
        ) for img_id in img_ids
    ]

image_input = torch.tensor(np.stack(images)).to(device)
image_features = model.encode_image(image_input)

In [34]:
image_features

tensor([[ 0.2434, -0.2942,  0.4368,  ...,  0.7529,  0.1901, -0.2384],
        [ 0.2539, -0.2722,  0.5786,  ...,  0.7734, -0.1420, -0.5156],
        [ 0.0046, -0.6860,  0.1312,  ...,  1.1094, -0.0214, -0.0644],
        ...,
        [ 0.0394, -0.3428,  0.4075,  ...,  0.8989,  0.2094, -0.3184],
        [-0.0790, -0.4011,  0.3728,  ...,  0.7495,  0.2615, -0.0906],
        [ 0.1292, -0.2399,  0.1924,  ...,  0.5698,  0.0909, -0.0884]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>)

In [35]:
image_features.shape

torch.Size([48, 512])

In [36]:
# combining two vectors
combo = torch.cat([text,image_features],dim = 1)

In [37]:
combo

tensor([[ 4.9408e+04,  5.5000e+02,  1.4880e+03,  ...,  7.5293e-01,
          1.9006e-01, -2.3840e-01],
        [ 4.9408e+04,  5.5000e+02,  1.4880e+03,  ...,  7.7344e-01,
         -1.4197e-01, -5.1562e-01],
        [ 4.9408e+04,  3.2000e+02,  4.3392e+04,  ...,  1.1094e+00,
         -2.1423e-02, -6.4392e-02],
        ...,
        [ 4.9408e+04,  2.4720e+04,  5.5320e+03,  ...,  8.9893e-01,
          2.0935e-01, -3.1836e-01],
        [ 4.9408e+04,  2.4720e+04,  5.5320e+03,  ...,  7.4951e-01,
          2.6147e-01, -9.0637e-02],
        [ 4.9408e+04,  1.5880e+04,  5.2500e+02,  ...,  5.6982e-01,
          9.0881e-02, -8.8379e-02]], device='cuda:0', dtype=torch.float16,
       grad_fn=<CatBackward0>)

In [38]:
combo.shape

torch.Size([48, 589])

In [39]:
# combo_embed = torch.nn.functional.normalize(combo)

In [40]:
combo_embedding = combo.cpu().detach().numpy()

## Random Forest Classifier

In [41]:
# imports
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# load data

# df = pd.read_csv(datafile_path)
# df["embedding"] = df.embedding.apply(eval).apply(np.array)  # convert string to array

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    combo_embedding, final_labels, test_size=0.2, random_state=42
)

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

                               precision    recall  f1-score   support

   Basal Cell Carcinoma (BCC)       0.40      1.00      0.57         2
                     Melanoma       0.75      0.60      0.67         5
  Merkel Cell Carcinoma (MCC)       0.00      0.00      0.00         1
Squamous Cell Carcinoma (SCC)       0.00      0.00      0.00         2

                     accuracy                           0.50        10
                    macro avg       0.29      0.40      0.31        10
                 weighted avg       0.45      0.50      0.45        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
from sklearn.svm import SVC ,SVR

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    combo_embedding, final_labels, test_size=0.2, random_state=42
)

svc_model = SVC(kernel='rbf', C = 2)
svc_model.fit(X_train, y_train)
# for validation data
preds = svc_model.predict(X_test)

report = classification_report(y_test, preds)
print(report)

                               precision    recall  f1-score   support

   Basal Cell Carcinoma (BCC)       0.29      1.00      0.44         2
                     Melanoma       1.00      0.60      0.75         5
  Merkel Cell Carcinoma (MCC)       0.00      0.00      0.00         1
Squamous Cell Carcinoma (SCC)       0.00      0.00      0.00         2

                     accuracy                           0.50        10
                    macro avg       0.32      0.40      0.30        10
                 weighted avg       0.56      0.50      0.46        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
text_embed = text.cpu().detach().numpy()


X_train, X_test, y_train, y_test = train_test_split(
    text_embed, final_labels, test_size=0.2, random_state=42
)

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

                               precision    recall  f1-score   support

   Basal Cell Carcinoma (BCC)       0.29      1.00      0.44         2
                     Melanoma       0.67      0.40      0.50         5
  Merkel Cell Carcinoma (MCC)       0.00      0.00      0.00         1
Squamous Cell Carcinoma (SCC)       0.00      0.00      0.00         2

                     accuracy                           0.40        10
                    macro avg       0.24      0.35      0.24        10
                 weighted avg       0.39      0.40      0.34        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
img_embed = image_features.cpu().detach().numpy()
X_train, X_test, y_train, y_test = train_test_split(
    img_embed, final_labels, test_size=0.2, random_state=42
)

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

                               precision    recall  f1-score   support

   Basal Cell Carcinoma (BCC)       0.17      0.50      0.25         2
                     Melanoma       1.00      0.40      0.57         5
  Merkel Cell Carcinoma (MCC)       0.00      0.00      0.00         1
Squamous Cell Carcinoma (SCC)       0.00      0.00      0.00         2

                     accuracy                           0.30        10
                    macro avg       0.29      0.23      0.21        10
                 weighted avg       0.53      0.30      0.34        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# LSTM Model

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import torch.nn.functional as F


mp_labels_2_num = {"Basal Cell Carcinoma (BCC)":0,"Squamous Cell Carcinoma (SCC)":1,"Melanoma":2, "Merkel Cell Carcinoma (MCC)":3}

final_labels_2 = [mp_labels_2_num[x] for x in final_labels]

X_train, X_test, y_train, y_test = train_test_split(
    combo_embedding, final_labels_2, test_size=0.2, random_state=42
)


class cancerDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        #return torch.from_numpy(self.X[idx].astype(np.int32)), self.y[idx] #, self.X[idx]
        return torch.from_numpy(self.X[idx]), self.y[idx]
train_ds = cancerDataset(X_train, y_train)
valid_ds = cancerDataset(X_test, y_test)


batch_size = 10
vocab_size = 589  #len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)





def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y in train_dl:
            # x = x.type(torch.float32)
            # y = y.type(torch.float32)
            # print(x.dtype)

            x = x.long()
            y = y.double()
            print(x.shape)
            print(y.shape)
            y_pred = []
            for xi in x:
              y_pred.append(model(x))
            # print(y_pred.shape)
            # y_pred = torch.max(y_pred, 1)[1]
            y_pred = torch.Tensor(y_pred)
            y_pred = y_pred.double()
            print(f"ypred {y_pred}. yactual {y}")
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y in valid_dl:
        # x = x.long()
        # y = y.long()
        y_hat = model(x)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total



class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True).type(torch.float32)
        self.linear = nn.Linear(hidden_dim, 4).type(torch.float32)
        
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        print("IN FOrward")
        x = x.type(torch.float32)

        print(x.shape)
        print(x.dtype)
        print(self.linear.weight.dtype)
        # x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        print(ht)
        return torch.argmax(self.linear(ht[-1]))


model_fixed =  LSTM_fixed_len(589, 589, 50)

train_model(model_fixed, epochs=5, lr=0.01)


In [ ]:
# BATCH_SIZE = 10
# ln = len(img_ids)

# results = []

# for i in tqdm(range(0,ln,BATCH_SIZE)):
#     images = [
#         preprocess(
#             Image.open(f"/content/{img_id}.jpg")
#         ) for img_id in img_ids[i:i+BATCH_SIZE]
#     ]
#     image_input = torch.tensor(np.stack(images)).to(device)
#     with torch.no_grad():
#         image_features = model.encode_image(image_input)
#         logits_per_image, logits_per_text = model(image_input, text)
#         # The softmax function takes the original confidence and applys a transform to make all the confidence add up to one
#         probs = logits_per_image.softmax(dim=-1).cpu().numpy()
#         results.append(probs)

# res = np.concatenate(results,axis=0)
# choices = np.argmax(res,axis=1)
# print(choices.shape)
# getlabel = lambda x:labels[x]
# vgetlabel = np.vectorize(getlabel)
# times = vgetlabel(choices)
# print(times)